<a href="https://colab.research.google.com/github/ArsheenSingh/Next-Word-Prediction-Model-using-a-Transformer-based-Architecture/blob/main/Next_word_predictor_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Oxford English Text.txt to Oxford English Text.txt


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np


In [ ]:
 file= open("Oxford English Text.txt","r",encoding="utf8")

 #Step1:store file in list
 Lines=[]
for i in file:
  Lines.append(i)

#Step2: Now, Convert list to string
for i in Lines:
  data= " ".join(Lines)

#Step3: Removing Unnecessary special words and spaces and all
data= data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('""',' ')
#Step 4: removing  unnecessary spaces
data = data.split()
data = ' '.join(data)

In [ ]:
data[:200]

'The Project Gutenberg eBook of The Oxford Book of American Essays This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no rest'

In [ ]:
len(data)

951276

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import os

In [ ]:
#Applying Tokenization
tokenize = Tokenizer()
tokenize.fit_on_texts([data])

# using pickle library to save the tokenizer for predict function
pickle.dump(tokenize, open('tokenizer1.pkl', 'wb'))

sequence_data = tokenize.texts_to_sequences([data])[0]
# printing the first 15 sequence data
sequence_data[:15]

[1, 192, 454, 1266, 2, 1, 1641, 362, 2, 104, 1012, 27, 1266, 7, 14]

In [ ]:
len(sequence_data)

168558

In [ ]:
#Determining the size of Vocablary(No of Unique words), +1 because index 0 will be reserved for padding
vocab_size = len(tokenize.word_index) + 1
print(vocab_size)

16121


In [ ]:
#using 3 words to predict the next word , here we will be inserting 4 words in the sequences list which is intially empty,in which 3 words will be my input and last word will be my output.
sequences = []
for i in range(3, len(sequence_data)):
  words = sequence_data[i-3:i+1]
  sequences.append(words)
print("The Length of sequences are: ", len(sequences))
#converting out list (sequences) in array
sequences = np.array(sequences)

The Length of sequences are:  168555


In [ ]:
sequences[:20]

array([[   1,  192,  454, 1266],
       [ 192,  454, 1266,    2],
       [ 454, 1266,    2,    1],
       [1266,    2,    1, 1641],
       [   2,    1, 1641,  362],
       [   1, 1641,  362,    2],
       [1641,  362,    2,  104],
       [ 362,    2,  104, 1012],
       [   2,  104, 1012,   27],
       [ 104, 1012,   27, 1266],
       [1012,   27, 1266,    7],
       [  27, 1266,    7,   14],
       [1266,    7,   14,    1],
       [   7,   14,    1,  235],
       [  14,    1,  235,    2],
       [   1,  235,    2,  667],
       [ 235,    2,  667, 1496],
       [   2,  667, 1496,    6],
       [ 667, 1496,    6,    1],
       [1496,    6,    1,  203]])

In [ ]:
#Seperating Our Indipendent(input) and Dependent (Output) Variables
X = []
y = []
for i in sequences:
  X.append(i[0:3]) #first 3 words in input
  y.append(i[3]) #Last word in output

X=np.array(X)
y=np.array(y)

In [ ]:
print("Data: ", X[:5])
print("Response: ", y[:])

Data:  [[   1  192  454]
 [ 192  454 1266]
 [ 454 1266    2]
 [1266    2    1]
 [   2    1 1641]]
Response:  [1266    2    1 1641  362]


In [ ]:
#Converting Class vectors into Binary class matrix (converting integer values into binary matrix reprentation) this is required as we will use loss function as categorical_cross_entropy is requires binary matrix representation
y= to_categorical(y, num_classes=vocab_size)
y[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
#Creating The LSTM Model
model= Sequential()
model.add(Embedding(vocab_size, 10))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))# relu will convert negative values to positive
model.add(Dense(vocab_size, activation="softmax"))

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#Saving the best model and weights using checkpoint library
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.keras", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=10, batch_size=16, callbacks=[checkpoint])

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.keras')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""

  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break

  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text = input("Enter your line: ")

  if text == "0":
      print("Execution completed.....")
      break

  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)

          Predict_Next_Words(model, tokenizer, text)

      except Exception as e:
        print("Error occurred: ",e)
        continue